<a href="https://colab.research.google.com/github/lokwq/TextBrewer/blob/add_note_examples/sst2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how to fine-tune a model on sst-2 dataset and how to distill the model with TextBrewer.

Detailed Docs can be find here:
https://github.com/airaria/TextBrewer

In [1]:
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'

In [2]:
import os
import torch
from transformers import BertForSequenceClassification, BertTokenizer,BertConfig, AutoModelForSequenceClassification,RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,load_metric
from functools import partial
from predict_function import predict

In [3]:
# Settings 
task_name = "mnli"
base_model_name = 'roberta-base'

### Prepare dataset to train

In [4]:
train_dataset = load_dataset('glue', 'mnli', split='train')#,cache_dir="/work/mhessent/cache")
val_dataset = load_dataset('glue', 'mnli', split='validation_matched')#,cache_dir="/work/mhessent/cache")
val_mm_dataset = load_dataset('glue', 'mnli', split='validation_mismatched')
test_dataset = load_dataset('glue', 'mnli', split='test_matched')#,cache_dir="/work/mhessent/cache")

Reusing dataset glue (/home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [5]:
train_dataset = train_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_dataset = val_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_mm_dataset = val_mm_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
test_dataset = test_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

val_dataset = val_dataset.remove_columns(['label'])
val_mm_dataset = val_mm_dataset.remove_columns(['label'])
test_dataset = test_dataset.remove_columns(['label'])
train_dataset = train_dataset.remove_columns(['label'])

Loading cached processed dataset at /home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-b84661f5fe95e1e2.arrow
Loading cached processed dataset at /home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-43d59c952978c460.arrow
Loading cached processed dataset at /home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-cce2791de0a64e6f.arrow
Loading cached processed dataset at /home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d933186d36b8e3ab.arrow


In [6]:
#model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
#tokenizer = RobertaTokenizer.from_pretrained("/work/mhessent/master_thesis/eval_out/roberta-base/mnli/lr3e-05_bs32_epochs10/checkpoint-49088")

In [7]:
MAX_LENGTH = 128
train_dataset = train_dataset.map(lambda e: tokenizer(e['premise'],e['hypothesis'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
val_dataset = val_dataset.map(lambda e: tokenizer(e['premise'],e['hypothesis'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
val_mm_dataset = val_mm_dataset.map(lambda e: tokenizer(e['premise'],e['hypothesis'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
test_dataset = test_dataset.map(lambda e: tokenizer(e['premise'],e['hypothesis'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)

Loading cached processed dataset at /home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-63d98290670a9a33.arrow
Loading cached processed dataset at /home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3b3d6e5eb30d55e3.arrow
Loading cached processed dataset at /home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-53c6e076a5990ca7.arrow
Loading cached processed dataset at /home/mhessent/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-25162f32847fd610.arrow


In [8]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_mm_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [9]:
train_dataset[:]["labels"].unique()

tensor([0, 1, 2])

In [10]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    metric = load_metric("glue","mnli")
    return metric.compute(predictions=preds, references=labels)



In [11]:
#start training 
"""
training_args = TrainingArguments(
    output_dir='outputs/results',          #output directory
    learning_rate=2e-5,
    num_train_epochs=3,              
    per_device_train_batch_size=32,                #batch size per device during training
    per_device_eval_batch_size=32,                #batch size for evaluation
    logging_dir='outputs/logs',            
    logging_steps=100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=True,
    # eval_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
    compute_metrics=compute_metrics
)

train_out = trainer.train()
"""
#after training, you could find traing logs and checpoints in your own dirve. also you can reset the file address in training args

'\ntraining_args = TrainingArguments(\n    output_dir=\'outputs/results\',          #output directory\n    learning_rate=2e-5,\n    num_train_epochs=3,              \n    per_device_train_batch_size=32,                #batch size per device during training\n    per_device_eval_batch_size=32,                #batch size for evaluation\n    logging_dir=\'outputs/logs\',            \n    logging_steps=100,\n    do_train=True,\n    do_eval=True,\n    no_cuda=False,\n    load_best_model_at_end=True,\n    # eval_steps=100,\n    evaluation_strategy="epoch",\n    save_strategy="epoch"\n)\n\ntrainer = Trainer(\n    model=model,                         \n    args=training_args,                  \n    train_dataset=train_dataset,         \n    eval_dataset=val_dataset,            \n    compute_metrics=compute_metrics\n)\n\ntrain_out = trainer.train()\n'

In [12]:
#torch.save(model.state_dict(), 'outputs/mnli_roberta_teacher_model.pt')


### Start distillation

In [13]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128) #prepare dataloader

In [14]:
import textbrewer
from textbrewer import GeneralDistiller
from textbrewer import TrainingConfig, DistillationConfig
from transformers import BertForSequenceClassification, BertConfig, AdamW,BertTokenizer, RobertaConfig, RobertaForSequenceClassification
from transformers import get_linear_schedule_with_warmup

import textbrewer
from textbrewer import GeneralDistiller
from textbrewer import TrainingConfig, DistillationConfig
from transformers import BertForSequenceClassification, BertConfig, AdamW,BertTokenizer, RobertaConfig, RobertaForSequenceClassification
from transformers import get_linear_schedule_with_warmupInitialize the student model by BertConfig and prepare the teacher model.

bert_config_L3.json refers to a 3-layer Bert.

bert_config.json refers to a standard 12-layer Bert.

In [15]:
#hub_model = RobertaForSequenceClassification.from_pretrained("/work/mhessent/master_thesis/eval_out/roberta-base/mnli/lr2e-05_bs32_epochs10/checkpoint-49088")
#torch.save(hub_model.state_dict(), 'outputs/hub_roberta_mnli_teacher_model.pt')

In [16]:
RobertaConfig.from_pretrained("roberta-base", output_hidden_states=True)

RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [17]:
config = RobertaConfig.from_pretrained("roberta-base", output_hidden_states=True)
#bert_config = BertConfig.from_json_file('/work/mhessent/TextBrewer/examples/student_config/bert_base_cased_config/bert_config.json')
config.output_hidden_states = True
config.num_labels = 3
teacher_model = RobertaForSequenceClassification(config) #, num_labels = 2
#teacher_model.load_state_dict(torch.load('outputs/mnli_teacher_model.pt'))
teacher_model.load_state_dict(torch.load('outputs/hub_roberta_mnli_teacher_model.pt'))
                             
"""
model = BertForSequenceClassification.from_pretrained("/work/mhessent/master_thesis/eval_out/bert-base-uncased/mnli/lr3e-05_bs32_epochs3/checkpoint-36816")
torch.save(model.state_dict(), 'outputs/hub_mnli_teacher_model.pt')
bert_config = BertConfig.from_json_file('/work/mhessent/TextBrewer/examples/student_config/bert_base_cased_config/bert_config.json')
bert_config.output_hidden_states = True
bert_config.vocab_size = 30522
bert_config.num_labels = 3
teacher_model = BertForSequenceClassification(bert_config) #, num_labels = 2
teacher_model.load_state_dict(torch.load('outputs/hub_mnli_teacher_model.pt'))
"""


teacher_model = teacher_model.to(device=device)



student_config = RobertaConfig.from_pretrained("roberta-base", output_hidden_states=True)
student_config.output_hidden_states = True
student_config.num_labels = 3
student_config.num_hidden_layers = 3
#student_config.vocab_size = teacher_model.config.vocab_size

continue_training = False
student_model = RobertaForSequenceClassification(student_config)
if continue_training:
    student_model.load_state_dict(torch.load('/work/mhessent/TextBrewer/examples/notebook_examples/saved_models/gs490880.pkl'))
student_model = student_model.to(device=device)


print(teacher_model.config.vocab_size)
print(student_model.config.vocab_size)
print(len(tokenizer))

50265
50265
50265


In [18]:

from torch.utils.data import DataLoader
eval_dataloader = DataLoader(val_dataset, batch_size=8)
from textbrewer.distiller_utils import move_to_device

metric= load_metric("glue","mnli")
#teacher_model.cpu()
teacher_model.to(device)
teacher_model.eval()
for batch in eval_dataloader:
    batch = {k: v for k, v in batch.items()}
    batch = move_to_device(batch,device)
    with torch.no_grad():
        outputs = teacher_model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()


{'accuracy': 0.8712175241976566}

The cell below is to distill the teacher model to student model you prepared.

After the code execution is complete, the distilled model will be in 'saved_model' in colab file list

In [ ]:
num_epochs = 60
num_training_steps = len(train_dataloader) * num_epochs
# Optimizer and learning rate scheduler
optimizer = AdamW(student_model.parameters(), lr=1e-4)

scheduler_class = get_linear_schedule_with_warmup
# arguments dict except 'optimizer'
scheduler_args = {'num_warmup_steps':int(0.1*num_training_steps), 'num_training_steps':num_training_steps}


def simple_adaptor(batch, model_outputs):
    return {'logits': model_outputs.logits, 'hidden': model_outputs.hidden_states}


from matches import matches
intermediate_matches = None
match_list_L4t = ["L4t_hidden_mse", "L4_hidden_smmd"]
match_list_L3 = ["L3_hidden_mse", "L3_hidden_smmd"]
intermediate_matches = []
for match in match_list_L3:
    intermediate_matches += matches[match]

output_dir = "outputs/" + base_model_name + "/" + task_name + "/" + "hl"+ str(student_model.config.num_hidden_layers) + "_hs" +  str(student_model.config.hidden_size) + "/"
distill_config = DistillationConfig()
    #,intermediate_matches=intermediate_matches)
train_config = TrainingConfig(device=device, output_dir = output_dir + "models/")



task_name = "mnli"
local_rank = -1
predict_batch_size = 32
device = device
do_train_eval = True

eval_datasets = [val_dataset,val_mm_dataset]

callback_func = partial(predict, eval_datasets=eval_datasets, output_dir=output_dir+"results/",task_name=task_name,local_rank=local_rank,predict_batch_size=predict_batch_size,device=device, do_train_eval=do_train_eval, train_dataset=train_dataset)

distiller = GeneralDistiller(
    train_config=train_config, distill_config=distill_config,
    model_T=teacher_model, model_S=student_model, 
    adaptor_T=simple_adaptor, adaptor_S=simple_adaptor)


with distiller:
    distiller.train(optimizer, train_dataloader, num_epochs, scheduler_class=scheduler_class, scheduler_args = scheduler_args, callback=callback_func)

/work/mhessent/miniconda/envs/thesis_test/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
2022/04/05 21:51:48 - INFO - Distillation -  Training steps per epoch: 3068
2022/04/05 21:51:48 - INFO - Distillation -  Checkpoints(step): [0]
2022/04/05 21:51:48 - INFO - Distillation -  Epoch 1
2022/04/05 21:51:48 - INFO - Distillation -  Length of current epoch in forward batch: 3068
2022/04/05 21:52:25 - INFO - Distillation -  Global step: 153, epoch step:153
2022/04/05 21:53:03 - INFO - Distillation -  Global step: 306, epoch step:306
2022/04/05 21:53:42 - INFO - Distillation -  Global step: 459, epoch step:459
2022/04/05 21:54:20 - INFO - Distillation -  Global step: 612, epoch step:612
2022/04/05 21:54:58 - INFO - Distillation -  Global step: 76

In [ ]:
test_model = RobertaForSequenceClassification(student_config)
test_model.load_state_dict(torch.load('/work/mhessent/TextBrewer/examples/notebook_examples/saved_models/gs39884.pkl'))#gs4210 is the distilled model weights file

In [ ]:
from torch.utils.data import DataLoader
eval_dataloader = DataLoader(val_dataset, batch_size=8)

In [ ]:
from textbrewer.distiller_utils import move_to_device

In [ ]:
metric= load_metric("accuracy")
test_model.to(device)
test_model.eval()
for batch in train_dataloader:
    batch = {k: v for k, v in batch.items()}
    batch = move_to_device(batch,device)
    with torch.no_grad():
        outputs = test_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()